In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
data = pd.read_csv("RPL.csv", encoding = 'cp1251', delimiter=';')
data.head()

,Часть,Год,Команда,Соперник,Минуты,Схема,Забито,Пропущено,Удары,Удары в створ,Передачи,Точные передачи,Навесы,Точные навесы,Владение,xG,PPDA,Победитель,Проигравший
0,1,2018,Ахмат,Арсенал,95,4-2-3-1 (100.0%),2,0,19,8,422,361,10,3,52.64,3.19,6.36,Ахмат,Арсенал
1,1,2018,Ахмат,Енисей,94,3-5-2 (100.0%),1,1,7,2,347,247,10,2,37.84,0.19,19.44,Ничья,Ничья
2,1,2018,Ахмат,ЦСКА,96,5-3-2 (79.58%),0,2,11,3,375,291,9,1,46.41,1.02,9.97,ЦСКА,Ахмат
3,1,2018,Ахмат,КС,94,5-3-2 (100.0%),2,1,8,2,455,372,14,6,52.39,0.95,9.19,Ахмат,КС
4,1,2018,Ахмат,Зенит,94,5-3-2 (93.08%),0,1,11,3,440,357,16,4,48.79,0.62,15.52,Зенит,Ахмат


In [3]:
RPL_2018_2019 = pd.read_csv('Team Name 2018 2019.csv', encoding = 'cp1251')

teamList = RPL_2018_2019['Team Name'].tolist()
teamList

['Анжи',
 'Ахмат',
 'Зенит',
 'Краснодар',
 'Локомотив',
 'Ростов',
 'Рубин',
 'Спартак',
 'Урал',
 'Уфа',
 'ЦСКА']

Удаляем матчи с командами, которые не участвуют в сезоне 2018/2019:

In [4]:
deleteTeam = [x for x in pd.unique(data['Соперник']) if x not in teamList]
for name in deleteTeam:
    data = data[data['Команда'] != name]
    data = data[data['Соперник'] != name]
data = data.reset_index(drop=True)

In [5]:
#Функция, возвращающая статистику команды за сезон 
def GetSeasonTeamStat(team, season):
    goalScored = 0 #Голов забито
    goalAllowed = 0 #Голов пропущено

    gameWin = 0 #Выиграно
    gameDraw = 0 #Ничья
    gameLost = 0 #Проиграно

    totalScore = 0 #Количество набранных очков

    matches = 0 #Количество сыгранных матчей
    
    xG = 0 #Ожидаемые голы
    
    shot = 0 #Удары
    shotOnTarget = 0 #Удары в створ
    
    cross = 0 #Навесы
    accurateCross = 0 #Точные навесы
    
    totalHandle = 0 #Владение мячом
    averageHandle = 0 #Среднее владение мячом за матч
    
    Pass = 0 #Пасы
    accuratePass = 0 #Точные пасы
    
    PPDA = 0 #Интенсивность прессинга в матче

    for i in range(len(data)):
        if (((data['Год'][i] == season) and (data['Команда'][i] == team) and (data['Часть'][i] == 2)) or ((data['Год'][i] == season-1) and (data['Команда'][i] == team) and (data['Часть'][i] == 1))):
            matches += 1
                
            goalScored += data['Забито'][i]
            goalAllowed += data['Пропущено'][i]

            if (data['Забито'][i] > data['Пропущено'][i]):
                totalScore += 3
                gameWin += 1
            elif (data['Забито'][i] < data['Пропущено'][i]):
                gameLost +=1
            else:
                totalScore += 1
                gameDraw += 1
            
            xG += data['xG'][i]
            
            shot += data['Удары'][i]
            shotOnTarget += data['Удары в створ'][i]
            
            Pass += data['Передачи'][i]
            accuratePass += data['Точные передачи'][i]
            
            totalHandle += data['Владение'][i]
            
            cross += data['Навесы'][i]
            accurateCross += data['Точные навесы'][i]
            
            PPDA += data['PPDA'][i]

    averageHandle = round(totalHandle/matches, 3) #Владение мячом в среднем за матч
    
    return [gameWin, gameDraw, gameLost, 
            goalScored, goalAllowed, totalScore, 
            round(xG, 3), round(PPDA, 3),
            shot, shotOnTarget, 
            Pass, accuratePass,
            cross, accurateCross,
            round(averageHandle, 3)]

In [6]:
GetSeasonTeamStat("Спартак", 2018) #Статистика Спартака за сезон 2017/2018 

[11, 4, 5, 39, 26, 37, 34.15, 178.24, 305, 115, 8978, 7364, 303, 119, 52.76]

In [7]:
returnNames = ["Выиграно", "Ничья", "Проиграно",
               "\nГолов забито", "Голов пропущено", "\nНабрано очков",
               "\nxG (за сезон)", "PPDA (за сезон)",
               "\nУдары", "Удары в створ", 
               "\nПасы", "Точные пасы",
               "\nНавесы", "Точные навесы",
                "\nВладение (в среднем за матч)"]

for i, n in zip(returnNames, GetSeasonTeamStat("Спартак", 2018)):
        print(i, n)

Выиграно 11
Ничья 4
Проиграно 5

Голов забито 39
Голов пропущено 26

Набрано очков 37

xG (за сезон) 34.15
PPDA (за сезон) 178.24

Удары 305
Удары в створ 115

Пасы 8978
Точные пасы 7364

Навесы 303
Точные навесы 119

Владение (в среднем за матч) 52.76


In [8]:
#Функция, возвращающая статистику всех команд за сезон
def GetSeasonAllTeamStat(season):
    annual = collections.defaultdict(list)
    for team in teamList:
        team_vector = GetSeasonTeamStat(team, season)
        annual[team] = team_vector
    return annual

Функция, которая будет возвращать обучающие данные. Она создает словарь с векторами команд за все сезоны. Для каждой игры функция рассчитывает разницу между векторами команд за определенный сезон и записывает в xTrain. Затем функция присваивает yTrain значение 1, если команда хозяев выигрывает, и 0 в противном случае.

In [9]:
#Получение тренировычных данных
def GetTrainingData(seasons):
    totalNumGames = 0
    for season in seasons:
        annual = data[data['Год'] == season]
        totalNumGames += len(annual.index)
    numFeatures = len(GetSeasonTeamStat('Зенит', 2016)) #случайная команда для определения размерности
    xTrain = np.zeros(( totalNumGames, numFeatures))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for season in seasons:
        team_vectors = GetSeasonAllTeamStat(season)
        annual = data[data['Год'] == season]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            team = row['Команда']
            t_vector = team_vectors[team]
            rivals = row['Соперник']
            r_vector = team_vectors[rivals]
            
            diff = [a - b for a, b in zip(t_vector, r_vector)]
            
            if len(diff) != 0:
                xTrainAnnual[counter] = diff
            if team == row['Победитель']:
                yTrainAnnual[counter] = 1
            else: 
                yTrainAnnual[counter] = 0
            counter += 1   
        xTrain[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain, yTrain

Поучаем обучающие данные за все сезоны с 2015/2016 по 2018/2019:

In [10]:
years = range(2016,2019)
xTrain, yTrain = GetTrainingData(years)

Для прогнозирования вероятности выигрыша будем использовать алгоритм машинного обучения LinearRegression из библиотеки Scikit-Learn

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
model = LinearRegression()
model.fit(xTrain, yTrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Функция, которая будет возвращать прогнозы. Она будет возвращать значение в промежутке от 0 до 1, где 0 — это проигрыш, а 1 — это выигрыш.

In [13]:
def createGamePrediction(team1_vector, team2_vector):
    diff = [[a - b for a, b in zip(team1_vector, team2_vector)]]
    predictions = model.predict(diff)
    return predictions

In [14]:
team1_name = "Зенит"
team2_name = "Спартак"

team1_vector = GetSeasonTeamStat(team1_name, 2019)
team2_vector = GetSeasonTeamStat(team2_name, 2019)

print ('Вероятность, что выиграет ' + team1_name + ':', createGamePrediction(team1_vector, team2_vector))
print ('Вероятность, что выиграет ' + team2_name + ':', createGamePrediction(team2_vector, team1_vector))

Вероятность, что выиграет Зенит: [0.47927615]
Вероятность, что выиграет Спартак: [0.24884885]


In [15]:
for team_name in teamList:
    team1_name = "ЦСКА"
    team2_name = team_name
    
    if(team1_name != team2_name):
        team1_vector = GetSeasonTeamStat(team1_name, 2019)
        team2_vector = GetSeasonTeamStat(team2_name, 2019)

        print(team1_name, createGamePrediction(team1_vector, team2_vector), " - ", team2_name, createGamePrediction(team2_vector, team1_vector,))

ЦСКА [0.57418478]  -  Анжи [0.15394022]
ЦСКА [0.46663608]  -  Ахмат [0.26148892]
ЦСКА [0.3644371]  -  Зенит [0.3636879]
ЦСКА [0.45534822]  -  Краснодар [0.27277678]
ЦСКА [0.50116796]  -  Локомотив [0.22695704]
ЦСКА [0.48567862]  -  Ростов [0.24244638]
ЦСКА [0.50959235]  -  Рубин [0.21853265]
ЦСКА [0.47965075]  -  Спартак [0.24847425]
ЦСКА [0.61386723]  -  Урал [0.11425777]
ЦСКА [0.49576571]  -  Уфа [0.23235929]
